In [1]:
import csv
import pandas as pd

a=[]
n = 0
with open("test.txt") as tsv:
    for line in csv.reader(tsv, dialect="excel-tab"):
        l = [x.replace('*', '').replace('+', '') for x in line[0].strip().split(' ') if x.replace('*', '').replace('+', '')]
        if n == 0:
            attrs = l[6].replace('(', '').replace(')', '').split(',')
            attrs = l[:5] + attrs
            print(attrs)
        else:
            l[1] = int(l[1][2])
            l[2] = int(l[2][2])
            attr_values = ''.join(l[5:]).replace('(', '').replace(')', '').split(',')
            l = l[:5]
            for value in attr_values:
                if value.isnumeric():
                    value = float(value)
                else:
                    value = value.replace("'", '')
                l.append(value)
            a.append(l)

        
        n+=1
        
df = pd.DataFrame(a, columns = attrs)

['inst#,', 'actual,', 'predicted,', 'error,', 'probability', 'Age', 'CompFreq', 'ConvertedComp', 'EdLevel', 'NEWOnboardGood', 'NEWOvertime', 'OrgSize', 'WorkWeekHrs', 'YearsCodePro', 'LookingForJob']


In [2]:
df['Correctly'] = (df['actual,'] == df['predicted,']).astype(int)

In [3]:
df

,"inst#,","actual,","predicted,","error,",probability,Age,CompFreq,ConvertedComp,EdLevel,NEWOnboardGood,NEWOvertime,OrgSize,WorkWeekHrs,YearsCodePro,LookingForJob,Correctly
0,1,1,1,0.119,0.881,24.0,Monthly,4944.0,Master/Doctoral,Yes,Rarely,20-99,45.0,5.0,No,1
1,2,1,1,0.397,0.603,25.0,Monthly,8400.0,University,Yes,Sometimes,20-99,42.0,1.0,Yes,1
2,3,1,1,0.223,0.777,26.0,Monthly,62424.0,University,Yes,Occasionally,1.000-4.999,40.0,8.0,NoButOpenToOpportunities,1
3,4,1,1,0.119,0.881,27.0,Monthly,78156.0,University,Yes,Often,100-499,40.0,3.0,No,1
4,5,1,1,0.223,0.777,31.504648,Monthly,108240.39508,University,Yes,Sometimes,20-99,8.0,6.0,NoButOpenToOpportunities,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6627,6628,1,1,0.119,0.881,27.0,Yearly,130000.0,University,Yes,Never,2-9,55.0,5.0,No,1
6628,6629,0,1,0.223,0.777,50.0,Yearly,127000.0,University,Yes,Often,10.000ormore,45.0,23.0,NoButOpenToOpportunities,0
6629,6630,1,1,0.119,0.881,25.0,Yearly,105000.0,University,Yes,Often,20-99,40.0,3.0,No,1
6630,6631,1,1,0.223,0.777,23.0,Yearly,25131.0,Master/Doctoral,Yes,Sometimes,10.000ormore,40.0,1.0,NoButOpenToOpportunities,1


In [35]:
# Reglas
TOTAL_INSTANCIAS = df.shape[0]
TOTAL_CLASE_1 = df['actual,'].sum()
TOTAL_CLASE_0 = TOTAL_INSTANCIAS - TOTAL_CLASE_1

def calculate_and_print(R, class_label):
    class_count = TOTAL_CLASE_1 if class_label == 1 else TOTAL_CLASE_0
    R_COUNT = R.shape[0]
    print('Clase: ' + str(class_label))
    print('Entradas: ' + str(R_COUNT))
    print('Bien Clasificados: ' + str(R['Correctly'].sum()))
    print('----------------------------------')
    print('Confianza: {} / {} = {:.4f} ({:.2f} %)'.format(R['Correctly'].sum(), R_COUNT, R['Correctly'].sum() / R_COUNT, (R['Correctly'].sum() / R_COUNT) * 100))
    print('Captura: {} / {} = {:.4f} ({:.2f} %)'.format(R['Correctly'].sum(), class_count, R['Correctly'].sum() / class_count, (R['Correctly'].sum() / class_count) * 100))
    print('Soporte: {} / {} = {:.4f} ({:.2f} %)'.format(R_COUNT, TOTAL_INSTANCIAS, R_COUNT / TOTAL_INSTANCIAS, (R_COUNT / TOTAL_INSTANCIAS) * 100))

In [36]:
print("Total Instancias Test: {}".format(TOTAL_INSTANCIAS))
print("Total Instancias Clase 0: {}".format(TOTAL_CLASE_0))
print("Total Instancias Clase 1: {}".format(TOTAL_CLASE_1))

Total Instancias Test: 6632
Total Instancias Clase 0: 1753
Total Instancias Clase 1: 4879


In [37]:
R = df.loc[(df['LookingForJob'] == "NoButOpenToOpportunities")]
calculate_and_print(R, class_label=1)

Clase: 1
Entradas: 3820
Bien Clasificados: 2786
----------------------------------
Confianza: 2786 / 3820 = 0.7293 (72.93 %)
Captura: 2786 / 4879 = 0.5710 (57.10 %)
Soporte: 3820 / 6632 = 0.5760 (57.60 %)


In [38]:
R = df.loc[(df['LookingForJob'] == "Yes") & (df['NEWOnboardGood'] == "Yes")]
calculate_and_print(R, class_label=1)

Clase: 1
Entradas: 357
Bien Clasificados: 199
----------------------------------
Confianza: 199 / 357 = 0.5574 (55.74 %)
Captura: 199 / 4879 = 0.0408 (4.08 %)
Soporte: 357 / 6632 = 0.0538 (5.38 %)


In [39]:
R = df.loc[df['LookingForJob'] == "No"]
calculate_and_print(R, class_label=1)

Clase: 1
Entradas: 2023
Bien Clasificados: 1780
----------------------------------
Confianza: 1780 / 2023 = 0.8799 (87.99 %)
Captura: 1780 / 4879 = 0.3648 (36.48 %)
Soporte: 2023 / 6632 = 0.3050 (30.50 %)


In [40]:
R = df.loc[(df['LookingForJob'] == "Yes") & (df['NEWOnboardGood'] == "No") & (df['OrgSize'] == "20-99")]
calculate_and_print(R, class_label=0)

Clase: 0
Entradas: 125
Bien Clasificados: 94
----------------------------------
Confianza: 94 / 125 = 0.7520 (75.20 %)
Captura: 94 / 1753 = 0.0536 (5.36 %)
Soporte: 125 / 6632 = 0.0188 (1.88 %)


In [41]:
R = df.loc[(df['LookingForJob'] == "Yes") & (df['NEWOnboardGood'] == "No") & (df['OrgSize'] == "10.000ormore")]
calculate_and_print(R, class_label=0)

Clase: 0
Entradas: 66
Bien Clasificados: 49
----------------------------------
Confianza: 49 / 66 = 0.7424 (74.24 %)
Captura: 49 / 1753 = 0.0280 (2.80 %)
Soporte: 66 / 6632 = 0.0100 (1.00 %)


In [42]:
R = df.loc[(df['LookingForJob'] == "Yes") & (df['NEWOnboardGood'] == "No") & (df['OrgSize'] == "2-9")]
calculate_and_print(R, class_label=0)

Clase: 0
Entradas: 32
Bien Clasificados: 20
----------------------------------
Confianza: 20 / 32 = 0.6250 (62.50 %)
Captura: 20 / 1753 = 0.0114 (1.14 %)
Soporte: 32 / 6632 = 0.0048 (0.48 %)


In [43]:
R = df.loc[(df['LookingForJob'] == "Yes") & (df['NEWOnboardGood'] == "No") & (df['OrgSize'] == "100-499")]
calculate_and_print(R, class_label=0)

Clase: 0
Entradas: 65
Bien Clasificados: 46
----------------------------------
Confianza: 46 / 65 = 0.7077 (70.77 %)
Captura: 46 / 1753 = 0.0262 (2.62 %)
Soporte: 65 / 6632 = 0.0098 (0.98 %)


In [44]:
R = df.loc[(df['LookingForJob'] == "Yes") & (df['NEWOnboardGood'] == "No") & (df['OrgSize'] == "500-999")]
calculate_and_print(R, class_label=0)

Clase: 0
Entradas: 31
Bien Clasificados: 21
----------------------------------
Confianza: 21 / 31 = 0.6774 (67.74 %)
Captura: 21 / 1753 = 0.0120 (1.20 %)
Soporte: 31 / 6632 = 0.0047 (0.47 %)


In [45]:
R = df.loc[(df['LookingForJob'] == "Yes") & (df['NEWOnboardGood'] == "No") & (df['OrgSize'] == "1.000-4.999")]
calculate_and_print(R, class_label=0)

Clase: 0
Entradas: 52
Bien Clasificados: 37
----------------------------------
Confianza: 37 / 52 = 0.7115 (71.15 %)
Captura: 37 / 1753 = 0.0211 (2.11 %)
Soporte: 52 / 6632 = 0.0078 (0.78 %)


In [46]:
R = df.loc[(df['LookingForJob'] == "Yes") & (df['NEWOnboardGood'] == "No") & (df['OrgSize'] == "5.000-9.999")]
calculate_and_print(R, class_label=0)

Clase: 0
Entradas: 10
Bien Clasificados: 9
----------------------------------
Confianza: 9 / 10 = 0.9000 (90.00 %)
Captura: 9 / 1753 = 0.0051 (0.51 %)
Soporte: 10 / 6632 = 0.0015 (0.15 %)


In [47]:
df["ConvertedComp"] = pd.to_numeric(df["ConvertedComp"], downcast="float")
R = df.loc[(df['LookingForJob'] == "Yes") & (df['NEWOnboardGood'] == "No") & (df['OrgSize'] == "10-19") & (df['ConvertedComp'] < 5660.5)]
calculate_and_print(R, class_label=1)

Clase: 1
Entradas: 12
Bien Clasificados: 4
----------------------------------
Confianza: 4 / 12 = 0.3333 (33.33 %)
Captura: 4 / 4879 = 0.0008 (0.08 %)
Soporte: 12 / 6632 = 0.0018 (0.18 %)


In [48]:
R = df.loc[(df['LookingForJob'] == "Yes") & (df['NEWOnboardGood'] == "No") & (df['OrgSize'] == "10-19") & (df['ConvertedComp'] >= 5660.5)]
calculate_and_print(R, class_label=0)

Clase: 0
Entradas: 39
Bien Clasificados: 29
----------------------------------
Confianza: 29 / 39 = 0.7436 (74.36 %)
Captura: 29 / 1753 = 0.0165 (1.65 %)
Soporte: 39 / 6632 = 0.0059 (0.59 %)
